CTGAN Model
===========

In this guide we will go through a series of steps that will let you
discover functionalities of the `CTGAN` model, including how to:

-   Create an instance of `CTGAN`.
-   Fit the instance to your data.
-   Generate synthetic versions of your data.
-   Use `CTGAN` to anonymize PII information.
-   Customize the data transformations to improve the learning process.
-   Specify hyperparameters to improve the output quality.

What is CTGAN?
--------------

The `sdv.tabular.CTGAN` model is based on the GAN-based Deep Learning
data synthesizer which was presented at the NeurIPS 2020 conference by
the paper titled [Modeling Tabular data using Conditional
GAN](https://arxiv.org/abs/1907.00503).

Let\'s now discover how to learn a dataset and later on generate
synthetic data with the same format and statistical properties by using
the `CTGAN` class from SDV.

Quick Usage
-----------

We will start by loading one of our demo datasets, the
`student_placements`, which contains information about MBA students that
applied for placements during the year 2020.

<div class="alert alert-warning">

**Warning**

In order to follow this guide you need to have `ctgan` installed on your
system. If you have not done it yet, please install `ctgan` now by
executing the command `pip install sdv` in a terminal.

</div>

In [2]:
from sdv.demo import load_tabular_demo

data = load_tabular_demo('student_placements')
data.head()

,student_id,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,17264,M,67.00,91.00,Commerce,58.00,Sci&Tech,False,0,55.0,Mkt&HR,58.80,27000.0,True,2020-07-23,2020-10-12,3.0
1,17265,M,79.33,78.33,Science,77.48,Sci&Tech,True,1,86.5,Mkt&Fin,66.28,20000.0,True,2020-01-11,2020-04-09,3.0
2,17266,M,65.00,68.00,Arts,64.00,Comm&Mgmt,False,0,75.0,Mkt&Fin,57.80,25000.0,True,2020-01-26,2020-07-13,6.0
3,17267,M,56.00,52.00,Science,52.00,Sci&Tech,False,0,66.0,Mkt&HR,59.43,NaN,False,NaT,NaT,NaN
4,17268,M,85.80,73.60,Commerce,73.30,Comm&Mgmt,False,0,96.8,Mkt&Fin,55.50,42500.0,True,2020-07-04,2020-09-27,3.0


As you can see, this table contains information about students which
includes, among other things:

-   Their id and gender
-   Their grades and specializations
-   Their work experience
-   The salary that they where offered
-   The duration and dates of their placement

You will notice that there is data with the following characteristics:

-   There are float, integer, boolean, categorical and datetime values.
-   There are some variables that have missing data. In particular, all
    the data related to the placement details is missing in the rows
    where the student was not placed.

Let us use `CTGAN` to learn this data and then sample synthetic data
about new students to see how well de model captures the characteristics
indicated above. In order to do this you will need to:

-   Import the `sdv.tabular.CTGAN` class and create an instance of it.
-   Call its `fit` method passing our table.
-   Call its `sample` method indicating the number of synthetic rows
    that you want to generate.

In [3]:
from sdv.tabular import CTGAN

model = CTGAN()
model.fit(data)

/Users/fealho/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass n_components=10 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
/Users/fealho/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass n_components=10 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
/Users/fealho/opt/anaconda3/lib/python3.8/site-packages/sklearn/mixture/_base.py:265: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  warnings.warn('Initialization %d did not converge. '
/Users/fealho/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass n_components=10 as keyword args. From versi

<div class="alert alert-info">

**Note**

Notice that the model `fitting` process took care of transforming the
different fields using the appropriate [Reversible Data
Transforms](http://github.com/sdv-dev/RDT) to ensure that the data has a
format that the underlying CTGANSynthesizer class can handle.

</div>

### Generate synthetic data from the model

Once the modeling has finished you are ready to generate new synthetic
data by calling the `sample` method from your model passing the number
of rows that we want to generate.

In [4]:
new_data = model.sample(200)

This will return a table identical to the one which the model was fitted
on, but filled with new data which resembles the original one.

In [5]:
new_data.head()

,student_id,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,17491,M,79.084456,42.879400,Science,66.284035,Sci&Tech,False,0,75.346292,Mkt&Fin,58.755787,NaN,True,2020-02-09,NaT,3.0
1,17478,F,83.416923,68.410226,Commerce,68.917519,Comm&Mgmt,False,0,73.437224,Mkt&Fin,74.568012,NaN,True,NaT,NaT,6.0
2,17317,F,77.622520,49.007200,Science,64.123089,Sci&Tech,False,0,70.768863,Mkt&Fin,68.071624,20728.093475,True,2020-03-03,NaT,3.0
3,17514,M,80.923513,68.483457,Science,52.172289,Comm&Mgmt,False,0,69.982091,Mkt&Fin,71.595540,NaN,False,2020-06-18,2020-07-26,12.0
4,17488,M,83.567888,96.960685,Commerce,63.184041,Sci&Tech,False,1,64.570397,Mkt&HR,72.648508,NaN,True,NaT,NaT,3.0


<div class="alert alert-info">

**Note**

You can control the number of rows by specifying the number of `samples`
in the `model.sample(<num_rows>)`. To test, try `model.sample(10000)`.
Note that the original table only had \~200 rows.

</div>

### Save and Load the model

In many scenarios it will be convenient to generate synthetic versions
of your data directly in systems that do not have access to the original
data source. For example, if you may want to generate testing data on
the fly inside a testing environment that does not have access to your
production database. In these scenarios, fitting the model with real
data every time that you need to generate new data is feasible, so you
will need to fit a model in your production environment, save the fitted
model into a file, send this file to the testing environment and then
load it there to be able to `sample` from it.

Let\'s see how this process works.

#### Save and share the model

Once you have fitted the model, all you need to do is call its `save`
method passing the name of the file in which you want to save the model.
Note that the extension of the filename is not relevant, but we will be
using the `.pkl` extension to highlight that the serialization protocol
used is [pickle](https://docs.python.org/3/library/pickle.html).

In [6]:
model.save('my_model.pkl')

This will have created a file called `my_model.pkl` in the same
directory in which you are running SDV.

<div class="alert alert-info">

**Important**

If you inspect the generated file you will notice that its size is much
smaller than the size of the data that you used to generate it. This is
because the serialized model contains **no information about the
original data**, other than the parameters it needs to generate
synthetic versions of it. This means that you can safely share this
`my_model.pkl` file without the risc of disclosing any of your real
data!

</div>

#### Load the model and generate new data

The file you just generated can be send over to the system where the
synthetic data will be generated. Once it is there, you can load it
using the `CTGAN.load` method, and then you are ready to sample new data
from the loaded instance:

In [7]:
loaded = CTGAN.load('my_model.pkl')
new_data = loaded.sample(200)

<div class="alert alert-warning">

**Warning**

Notice that the system where the model is loaded needs to also have
`sdv` and `ctgan` installed, otherwise it will not be able to load the
model and use it.

</div>

### Specifying the Primary Key of the table

One of the first things that you may have noticed when looking that demo
data is that there is a `student_id` column which acts as the primary
key of the table, and which is supposed to have unique values. Indeed,
if we look at the number of times that each value appears, we see that
all of them appear at most once:

In [8]:
data.student_id.value_counts().max()

1

However, if we look at the synthetic data that we generated, we observe
that there are some values that appear more than once:

In [9]:
new_data[new_data.student_id == new_data.student_id.value_counts().index[0]]

,student_id,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
94,17528,M,59.944795,77.944853,Commerce,57.206106,Comm&Mgmt,False,0,59.565290,Mkt&HR,77.386554,NaN,True,NaT,2020-08-26,3.0
107,17528,M,74.721451,58.133376,Arts,46.812296,Comm&Mgmt,False,0,59.368609,Mkt&Fin,79.897333,NaN,True,2020-02-01,2020-08-25,3.0
122,17528,F,87.986033,40.398743,Commerce,48.812232,Comm&Mgmt,False,0,77.680702,Mkt&Fin,67.051164,16589.401754,True,2020-03-07,NaT,NaN
145,17528,M,63.336766,46.616861,Commerce,50.129074,Others,True,0,58.424954,Mkt&Fin,62.106069,NaN,True,2020-01-28,2020-07-21,12.0


This happens because the model was not notified at any point about the
fact that the `student_id` had to be unique, so when it generates new
data it will provoke collisions sooner or later. In order to solve this,
we can pass the argument `primary_key` to our model when we create it,
indicating the name of the column that is the index of the table.

In [10]:
model = CTGAN(
    primary_key='student_id'
)
model.fit(data)
new_data = model.sample(200)
new_data.head()

/Users/fealho/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass n_components=10 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
/Users/fealho/opt/anaconda3/lib/python3.8/site-packages/sklearn/mixture/_base.py:265: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  warnings.warn('Initialization %d did not converge. '
/Users/fealho/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass n_components=10 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
/Users/fealho/opt/anaconda3/lib/python3.8/site-packages/sklearn/mixture/_base.py:265: ConvergenceWarning: Initialization 1 did not converge. Try differ

,student_id,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,0,M,52.857416,65.609276,Science,54.082215,Comm&Mgmt,False,1,68.884411,Mkt&Fin,53.88515,28127.194616,True,2020-03-06,2020-08-21,NaN
1,1,M,91.990401,72.383532,Commerce,60.587298,Sci&Tech,False,0,59.038902,Mkt&Fin,70.62782,26946.985840,True,NaT,2020-09-25,NaN
2,2,M,63.821491,63.873259,Commerce,65.565877,Comm&Mgmt,True,0,44.527151,Mkt&HR,64.78228,NaN,True,2019-12-31,2020-08-26,3.0
3,3,M,96.492221,76.393542,Commerce,76.091567,Comm&Mgmt,True,0,56.627190,Mkt&Fin,67.44346,31402.465712,False,2019-12-29,2020-09-17,3.0
4,4,M,78.794877,80.031310,Arts,51.119257,Comm&Mgmt,False,0,48.044888,Mkt&HR,60.26091,29256.271471,True,NaT,2020-08-07,NaN


As a result, the model will learn that this column must be unique and
generate a unique sequence of values for the column:

In [11]:
new_data.student_id.value_counts().max()

1

### Anonymizing Personally Identifiable Information (PII)

There will be many cases where the data will contain Personally
Identifiable Information which we cannot disclose. In these cases, we
will want our Tabular Models to replace the information within these
fields with fake, simulated data that looks similar to the real one but
does not contain any of the original values.

Let\'s load a new dataset that contains a PII field, the
`student_placements_pii` demo, and try to generate synthetic versions of
it that do not contain any of the PII fields.

<div class="alert alert-info">

**Note**

The `student_placements_pii` dataset is a modified version of the
`student_placements` dataset with one new field, `address`, which
contains PII information about the students. Notice that this additional
`address` field has been simulated and does not correspond to data from
the real users.

</div>

In [12]:
data_pii = load_tabular_demo('student_placements_pii')
data_pii.head()

,student_id,address,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,17264,"70304 Baker Turnpike\nEricborough, MS 15086",M,67.00,91.00,Commerce,58.00,Sci&Tech,False,0,55.0,Mkt&HR,58.80,27000.0,True,2020-07-23,2020-10-12,3.0
1,17265,"805 Herrera Avenue Apt. 134\nMaryview, NJ 36510",M,79.33,78.33,Science,77.48,Sci&Tech,True,1,86.5,Mkt&Fin,66.28,20000.0,True,2020-01-11,2020-04-09,3.0
2,17266,"3702 Bradley Island\nNorth Victor, FL 12268",M,65.00,68.00,Arts,64.00,Comm&Mgmt,False,0,75.0,Mkt&Fin,57.80,25000.0,True,2020-01-26,2020-07-13,6.0
3,17267,Unit 0879 Box 3878\nDPO AP 42663,M,56.00,52.00,Science,52.00,Sci&Tech,False,0,66.0,Mkt&HR,59.43,NaN,False,NaT,NaT,NaN
4,17268,"96493 Kelly Canyon Apt. 145\nEast Steven, NC 3...",M,85.80,73.60,Commerce,73.30,Comm&Mgmt,False,0,96.8,Mkt&Fin,55.50,42500.0,True,2020-07-04,2020-09-27,3.0


If we use our tabular model on this new data we will see how the
synthetic data that it generates discloses the addresses from the real
students:

In [13]:
model = CTGAN(
    primary_key='student_id',
)
model.fit(data_pii)
new_data_pii = model.sample(200)
new_data_pii.head()

/Users/fealho/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass n_components=10 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
/Users/fealho/opt/anaconda3/lib/python3.8/site-packages/sklearn/mixture/_base.py:265: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  warnings.warn('Initialization %d did not converge. '
/Users/fealho/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass n_components=10 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
/Users/fealho/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass n_components=10 as keyword args. From versi

,student_id,address,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,0,"8585 Jennifer Road Apt. 853\nNorth Paulside, T...",F,90.051947,83.548322,Commerce,69.606137,Comm&Mgmt,True,0,60.884157,Mkt&HR,63.634471,30218.876324,True,2020-09-05,NaT,NaN
1,1,"2707 Maria Parkways Apt. 743\nAlisonview, TN 8...",M,75.670294,66.799013,Commerce,84.792989,Sci&Tech,False,0,46.925344,Mkt&Fin,57.505432,NaN,True,2020-02-27,2020-09-24,6.0
2,2,"835 Patel Trace Apt. 213\nWoodtown, NY 30903",M,88.835466,62.198022,Science,71.866020,Comm&Mgmt,True,1,82.146699,Mkt&Fin,69.862420,31312.169631,False,2020-07-27,2020-06-25,NaN
3,3,"51290 Escobar Place\nSouth Madison, WI 89087",M,88.736972,46.525198,Science,70.998486,Sci&Tech,True,1,47.355710,Mkt&HR,61.947464,29732.206493,False,NaT,2020-09-26,NaN
4,4,"7488 Grant Vista\nPort David, CA 40814",M,76.357108,84.170176,Science,70.884952,Comm&Mgmt,False,0,72.595038,Mkt&Fin,63.272117,NaN,True,2020-04-05,2020-07-02,NaN


More specifically, we can see how all the addresses that have been
generated actually come from the original dataset:

In [14]:
new_data_pii.address.isin(data_pii.address).sum()

200

In order to solve this, we can pass an additional argument
`anonymize_fields` to our model when we create the instance. This
`anonymize_fields` argument will need to be a dictionary that contains:

-   The name of the field that we want to anonymize.
-   The category of the field that we want to use when we generate fake
    values for it.

The list complete list of possible categories can be seen in the [Faker
Providers](https://faker.readthedocs.io/en/master/providers.html) page,
and it contains a huge list of concepts such as:

-   name
-   address
-   country
-   city
-   ssn
-   credit_card_number
-   credit_card_expire
-   credit_card_security_code
-   email
-   telephone
-   \...

In this case, since the field is an e-mail address, we will pass a
dictionary indicating the category `address`

In [15]:
model = CTGAN(
    primary_key='student_id',
    anonymize_fields={
        'address': 'address'
    }
)
model.fit(data_pii)

/Users/fealho/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass n_components=10 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
/Users/fealho/opt/anaconda3/lib/python3.8/site-packages/sklearn/mixture/_base.py:265: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  warnings.warn('Initialization %d did not converge. '
/Users/fealho/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass n_components=10 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
/Users/fealho/opt/anaconda3/lib/python3.8/site-packages/sklearn/mixture/_base.py:265: ConvergenceWarning: Initialization 1 did not converge. Try differ

As a result, we can see how the real `address` values have been replaced
by other fake addresses:

In [16]:
new_data_pii = model.sample(200)
new_data_pii.head()

,student_id,address,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,0,"71195 Jamie Row\nWest Adam, VT 04183",F,58.462196,48.250242,Commerce,70.125988,Sci&Tech,True,0,75.266813,Mkt&Fin,62.892303,26289.425644,True,2020-04-08,2020-06-30,3.0
1,1,75994 Mcdonald Motorway Suite 874\nLake Christ...,M,79.109990,63.495240,Commerce,74.364447,Sci&Tech,True,1,74.278876,Mkt&Fin,75.503900,21645.598370,True,2020-03-05,2020-08-29,6.0
2,2,"18262 Melody Trail Apt. 187\nHousestad, HI 55300",F,57.421199,46.120778,Science,73.772790,Comm&Mgmt,True,0,102.533337,Mkt&Fin,67.391535,NaN,True,2020-02-22,2020-10-09,NaN
3,3,"78604 Kim Orchard Apt. 238\nRobertstown, PA 00557",F,77.520915,54.794390,Commerce,51.148910,Comm&Mgmt,True,0,62.554530,Mkt&Fin,68.122473,18781.892592,True,2020-04-02,2020-10-22,12.0
4,4,"2305 Mejia Ways Apt. 480\nSouth Curtis, GA 41164",M,49.827784,70.106479,Commerce,59.048126,Sci&Tech,False,0,65.212943,Mkt&Fin,67.387723,NaN,False,2020-03-25,2020-10-07,3.0


Which means that none of the original addresses can be found in the
sampled data:

In [17]:
data_pii.address.isin(new_data_pii.address).sum()

0

Advanced Usage
--------------

Now that we have discovered the basics, let\'s go over a few more
advanced usage examples and see the different arguments that we can pass
to our `CTGAN` Model in order to customize it to our needs.

### How to modify the CTGAN Hyperparameters?

A part from the common Tabular Model arguments, `CTGAN` has a number of
additional hyperparameters that control its learning behavior and can
impact on the performance of the model, both in terms of quality of the
generated data and computational time.

-   `epochs` and `batch_size`: these arguments control the number of
    iterations that the model will perform to optimize its parameters,
    as well as the number of samples used in each step. Its default
    values are `300` and `500` respectively, and `batch_size` needs to
    always be a value which is multiple of `10`. These hyperparameters
    have a very direct effect in time the training process lasts but
    also on the performance of the data, so for new datasets, you might
    want to start by setting a low value on both of them to see how long
    the training process takes on your data and later on increase the
    number to acceptable values in order to improve the performance.
-   `log_frequency`: Whether to use log frequency of categorical levels
    in conditional sampling. It defaults to `True`. This argument affects
    how the model processes the frequencies of the categorical values that
    are used to condition the rest of the values. In some cases, changing
    it to `False` could lead to better performance.
-   `embedding_dim` (int): Size of the random sample passed to the
    Generator. Defaults to 128.
-   `generator_dim` (tuple or list of ints): Size of the output samples for
    each one of the Residuals. A Resiudal Layer will be created for each
    one of the values provided. Defaults to (256, 256).
-   `discriminator_dim` (tuple or list of ints): Size of the output samples for
    each one of the Discriminator Layers. A Linear Layer will be created
    for each one of the values provided. Defaults to (256, 256).
-   `generator_lr` (float): Learning rate for the generator. Defaults to 2e-4.
-   `generator_decay` (float): Generator weight decay for the Adam Optimizer.
    Defaults to 1e-6.
-   `discriminator_lr` (float): Learning rate for the discriminator.
    Defaults to 2e-4.
-   `discriminator_decay` (float): Discriminator weight decay for the Adam
    Optimizer. Defaults to 1e-6.
-   `discriminator_steps` (int): Number of discriminator updates to do for
    each generator update. From the WGAN paper: https://arxiv.org/abs/1701.07875.
    WGAN paper default is 5. Default used is 1 to match original CTGAN
    implementation.
-   `verbose`: Whether to print fit progress on stdout. Defaults to
    `False`.

<div class="alert alert-warning">

**Warning**

Notice that the value that you set on the `batch_size` argument must
always be a multiple of `10`!

</div>

As an example, we will try to fit the `CTGAN` model slightly increasing
the number of epochs, reducing the `batch_size`, adding one additional
layer to the models involved and using a smaller wright decay.

Before we start, we will evaluate the quality of the previously
generated data using the `sdv.evaluation.evaluate` function

In [18]:
from sdv.evaluation import evaluate

evaluate(new_data, data)

/Users/fealho/opt/anaconda3/lib/python3.8/site-packages/sklearn/mixture/_base.py:265: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  warnings.warn('Initialization %d did not converge. '
/Users/fealho/opt/anaconda3/lib/python3.8/site-packages/sklearn/mixture/_base.py:265: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  warnings.warn('Initialization %d did not converge. '
/Users/fealho/opt/anaconda3/lib/python3.8/site-packages/sklearn/mixture/_base.py:265: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  warnings.warn('Initialization %d did not converge. '
/Users/fealho/opt/anaconda3/lib/python3.8/site-packages/sklearn/mixture/_base.py:265: ConvergenceWarning: Initialization 1 did not converge. Try differ

0.3153902510954942

Afterwards, we create a new instance of the `CTGAN` model with the
hyperparameter values that we want to use

In [19]:
model = CTGAN(
    primary_key='student_id',
    epochs=500,
    batch_size=100,
    generator_dim=(256, 256, 256),
    discriminator_dim=(256, 256, 256)
)

And fit to our data.

In [20]:
model.fit(data)

/Users/fealho/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass n_components=10 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
/Users/fealho/opt/anaconda3/lib/python3.8/site-packages/sklearn/mixture/_base.py:265: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  warnings.warn('Initialization %d did not converge. '
/Users/fealho/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass n_components=10 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
/Users/fealho/opt/anaconda3/lib/python3.8/site-packages/sklearn/mixture/_base.py:265: ConvergenceWarning: Initialization 1 did not converge. Try differ

Finally, we are ready to generate new data and evaluate the results.

In [21]:
new_data = model.sample(len(data))
evaluate(new_data, data)

/Users/fealho/opt/anaconda3/lib/python3.8/site-packages/sklearn/mixture/_base.py:265: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  warnings.warn('Initialization %d did not converge. '
/Users/fealho/opt/anaconda3/lib/python3.8/site-packages/sdmetrics/column_pairs/statistical/kl_divergence.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  real_data[pd.isnull(real_data)] = 0.0
/Users/fealho/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:3086: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing

0.2750368795263302

As we can see, in this case these modifications changed the obtained
results slightly, but they did neither introduce dramatic changes in the
performance.


### Conditional Sampling

As the name implies, conditional sampling allows us to sample from a conditional distribution using the `CTGAN` model, which means we can generate only values that satisfy certain conditions. These conditional values can be passed to the `conditions` parameter in the `sample` method either as a dataframe or a dictionary.

In case a dictionary is passed, the model will generate as many rows as requested, all of which will satisfy the specified conditions, such as `gender = M`.

In [22]:
conditions = {
    "gender": 'M'
}
model.sample(5, conditions=conditions)

,student_id,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,1,M,94.577592,60.496643,Commerce,89.173209,Comm&Mgmt,True,1,104.915987,Mkt&HR,70.347984,27807.887553,True,2020-06-17,2020-06-04,3.0
1,2,M,77.641056,29.066432,Science,76.834251,Sci&Tech,True,0,106.007703,Mkt&HR,58.770123,25964.549963,True,2020-01-07,2020-03-01,3.0
2,3,M,85.404647,47.673576,Science,76.895656,Sci&Tech,True,1,102.433259,Mkt&HR,82.364388,29649.056901,True,2020-06-03,2020-01-31,3.0
3,4,M,64.635844,41.198689,Science,86.686064,Comm&Mgmt,True,1,113.474699,Mkt&HR,51.215662,29858.421513,True,2019-12-28,2020-02-27,12.0
4,0,M,84.543415,51.915410,Science,50.760054,Sci&Tech,True,0,111.459041,Mkt&Fin,55.540671,21384.269243,True,2020-01-10,2020-08-30,NaN


It's also possible to condition on multiple columns, such as `gender = M, work_experience = True, high_spec = Science`.

In [23]:
conditions = {
    'gender': 'M',
    'work_experience': True,
    'high_spec': 'Science'
}
model.sample(5, conditions=conditions)

,student_id,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,0,M,93.920843,79.776642,Science,62.803702,Sci&Tech,True,0,89.136386,Mkt&Fin,67.293768,NaN,True,2020-03-05,2020-10-08,6.0
1,4,M,86.564210,37.907983,Science,97.314226,Sci&Tech,True,0,60.530553,Mkt&HR,49.814143,30123.798165,False,2019-12-13,2020-06-06,NaN
2,1,M,91.889555,30.662390,Science,75.729304,Comm&Mgmt,True,1,96.238651,Mkt&Fin,74.897840,25674.464837,False,2020-01-01,2020-06-28,12.0
3,4,M,100.108085,59.475979,Science,84.368116,Comm&Mgmt,True,0,110.705234,Mkt&HR,55.354024,21462.140603,True,2019-12-13,NaT,3.0
4,6,M,88.500442,48.479408,Science,88.064235,Comm&Mgmt,True,0,94.291124,Mkt&Fin,64.861359,25685.845598,False,2019-12-25,NaT,3.0


`conditions` can also be passed as a dataframe. In that case, the model will generate one sample for each row of the dataframe, sorted in the same order. Since the model already knows how many samples to generate, passing it as a parameter is unnecessary. For example, if we want to generate three samples where `gender = M` and three samples with `gender = F`, we can do the following: 

In [24]:
import pandas as pd 

conditions = pd.DataFrame({
    'gender': ['M', 'M', 'M', 'F', 'F', 'F'],
    'work_experience': [True, True, True, True, True, True]
})
model.sample(conditions=conditions)

,student_id,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,0,M,92.486729,35.257581,Commerce,76.432621,Comm&Mgmt,True,1,92.938413,Mkt&Fin,57.204752,23956.328993,True,2020-01-04,NaT,3.0
1,0,M,88.715184,61.167604,Commerce,95.619791,Comm&Mgmt,True,0,73.249674,Mkt&Fin,63.742348,27286.603104,True,2020-01-22,2021-01-16,NaN
2,2,M,47.430236,31.119407,Science,74.101852,Comm&Mgmt,True,1,81.801628,Mkt&Fin,66.292753,30342.559521,True,2020-01-07,2020-06-14,NaN
3,1,F,95.124210,46.919349,Arts,67.464898,Comm&Mgmt,True,0,79.779888,Mkt&HR,77.012577,24374.027609,True,2020-02-14,2020-08-17,3.0
4,2,F,85.217302,18.901712,Commerce,75.627358,Sci&Tech,True,1,75.592034,Mkt&HR,71.929251,33578.631744,True,2020-02-05,NaT,6.0
5,1,F,98.657065,59.306337,Commerce,76.590297,Comm&Mgmt,True,0,100.073312,Mkt&Fin,61.765921,30754.197238,True,2019-12-24,2020-07-03,6.0


`CTGAN` also supports conditioning on continuous values, as long as the values are within the range of seen numbers. For example, if all the values of the dataset are within 0 and 1, `CTGAN` will not be able to set this value to 1000.

In [29]:
conditions = {
    'degree_perc': 70.0
}
model.sample(5, conditions=conditions)

,student_id,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,17388,M,73.928445,53.939157,Commerce,70.0,Comm&Mgmt,False,0,51.994453,Mkt&Fin,49.446954,30793.879025,True,2020-01-08,NaT,12.0
1,17335,F,47.175276,34.868506,Commerce,70.0,Sci&Tech,True,0,64.870702,Mkt&HR,67.747288,27454.307890,True,2019-12-14,2020-06-12,12.0
2,17390,M,53.011181,71.794440,Commerce,70.0,Sci&Tech,True,1,60.512838,Mkt&Fin,66.126284,32527.736354,True,2020-02-18,NaT,NaN
3,17384,F,44.131180,43.917175,Science,70.0,Sci&Tech,False,0,78.736036,Mkt&Fin,50.824366,NaN,True,2020-07-11,2020-03-11,NaN
4,17338,M,81.287353,37.823967,Commerce,70.0,Comm&Mgmt,False,0,91.623221,Mkt&Fin,51.962894,30267.089598,True,NaT,2020-07-06,NaN


<div class="alert alert-info">

**Note**
Currently, conditional sampling works through a rejection sampling process, where rows are sampled repeatedly until one that satisfies the conditions is found. In case you are running into a `Could not get enough valid rows within x trials` or simply wish to optimize the results, there are three parameters that can be fine-tuned: max_rows_multiplier, max_retries and float_rtol. More information about these parameters can be found in the API section.


</div>

### How do I specify constraints?

If you look closely at the data you may notice that some properties were
not completely captured by the model. For example, you may have seen
that sometimes the model produces an `experience_years` number greater
than `0` while also indicating that `work_experience` is `False`. These
type of properties are what we call `Constraints` and can also be
handled using `SDV`. For further details about them please visit the
[Handling Constraints](04_Handling_Constraints.ipynb) guide.

### Can I evaluate the Synthetic Data?

A very common question when someone starts using **SDV** to generate
synthetic data is: *\"How good is the data that I just generated?\"*

In order to answer this question, **SDV** has a collection of metrics
and tools that allow you to compare the *real* that you provided and the
*synthetic* data that you generated using **SDV** or any other tool.

You can read more about this in the [Evaluating Synthetic Data Generators](
05_Evaluating_Synthetic_Data_Generators.ipynb) guide.